In [2]:
!pip install shap
!pip install umap-learn
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.6/572.6 kB 9.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 21.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82816 sha256=dba3fb5e10846961918e59ede0c931c594efcefa807cf014e037f201a611d357
  Stored in directory: /root/.cache/pip/wheels/a0/e8/c6/a37ea663620bd5200ea1ba0907ab3c217042c1d035ef606acc
  Created wheel for pynndescent: filename=pynndescent-0.5.10-py3-none-any.whl size=55622 sha256=3f23a73c3b5fb33f72606e586430308baf2d8348bd4f8786fcbe3c6f988fd48a
  Stored in directory: /root/.cache/pip/wheels/4a/38/5

In [1]:
import os
import pickle as pickle
import glob

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import lightgbm as lgbm
import shap

from scipy.spatial.transform import Rotation
import time
from joblib import Parallel, delayed
import umap


In [2]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import optuna  # pip install optuna
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
from optuna.integration import LightGBMPruningCallback

def objective(trial, X, y):
    param_grid = {
        # "device_type": trial.suggest_categorical("device_type", ['gpu']),
        "n_estimators": trial.suggest_categorical("n_estimators", [10000]),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "num_leaves": trial.suggest_int("num_leaves", 20, 3000, step=20),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 200, 10000, step=100),
        "lambda_l1": trial.suggest_int("lambda_l1", 0, 100, step=5),
        "lambda_l2": trial.suggest_int("lambda_l2", 0, 100, step=5),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 0, 15),
        "bagging_fraction": trial.suggest_float(
            "bagging_fraction", 0.2, 0.9, step=0.1
        ),
        "bagging_freq": trial.suggest_categorical("bagging_freq", [1]),
        "feature_fraction": trial.suggest_float(
            "feature_fraction", 0.2, 0.9, step=0.1
        ),
    }

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1121218)

    cv_scores = np.empty(5)
    for idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
        model = lgbm.LGBMClassifier(objective="binary", **param_grid)
        model.fit(
            X_train,
            y_train,
            eval_set=[(X_test, y_test)],
            eval_metric="binary_logloss",
            early_stopping_rounds=100,
            callbacks=[
                LightGBMPruningCallback(trial, "binary_logloss")
            ],  # Add a pruning callback
        )
        preds = model.predict_proba(X_test)
        cv_scores[idx] = log_loss(y_test, preds)

    return np.mean(cv_scores)

# study = optuna.create_study(direction="minimize", study_name="LGBM Classifier")
# func = lambda trial: objective(trial, X, y)
# study.optimize(func, n_trials=20)

# print(f"\tBest value (rmse): {study.best_value:.5f}")
# print(f"\tBest params:")

# for key, value in study.best_params.items():
#     print(f"\t\t{key}: {value}")
    
# -----------------------------------------------------
# Best value (binary_logloss): 0.35738
# 	Best params:
# 		device: gpu
# 		lambda_l1: 7.71800699380605e-05
# 		lambda_l2: 4.17890272377219e-06
# 		bagging_fraction: 0.7000000000000001
# 		feature_fraction: 0.4
# 		bagging_freq: 5
# 		max_depth: 5
# 		num_leaves: 1007
# 		min_data_in_leaf: 45
# 		min_split_gain: 15.703519227860273
# 		learning_rate: 0.010784015325759629
# 		n_estimators: 10000

In [ ]:
# Define the directory where the pickle files are located
pickle_directory = '/content/drive/My Drive/00_project/05_fly-arena/10_Locomotion/output_windows/all/'

dfs = []  # List to store individual dataframes

for filename in os.listdir(pickle_directory):
    print(f"Processing file: {filename}")
    if filename.endswith('.pkl'):
        # Extract the window size from the filename
        window_size = int(filename.split('_')[1])

        if window_size == 10:

            # Read the pickle file
            with open(os.path.join(pickle_directory, filename), 'rb') as f:
                data = pickle.load(f)
                
            # Select columns based on window_size
            columns = data.columns[:window_size * 4]
            # print(data.head())
            print(f"DataFrame selected... column size: {window_size * 4}")
            y = data.iloc[:, -1]
            X = data[columns]

            print(f"Running LGBM optimization using Optuna... column size: {window_size * 4}")

            study = optuna.create_study(direction="minimize", 
                                        study_name="LGBM Classifier", 
                                        sampler=optuna.samplers.RandomSampler())
            
            func = lambda trial: objective(trial, X, y)
            study.optimize(func, n_trials=30)

            fig = optuna.visualization.plot_optimization_history(study)
            fig.show()

            print(f"\tBest value (rmse): {study.best_value:.5f}")
            print(f"\tBest params:")

            for key, value in study.best_params.items():
                print(f"\t\t{key}: {value}")



        #     # fit a GBT model to the data
        #     m = lgbm.LGBMClassifier()
        #     m.fit(X, y)

        #     # compute SHAP values
        #     explainer = shap.Explainer(m)
        #     shap_values = explainer(X)
            
        # # compute 2D embedding of raw variable values
        #     X_2d = umap.UMAP(
        #     n_components=2, n_neighbors=200, min_dist=0
        #     ).fit_transform(X)

        #     # compute 2D embedding of SHAP values
        #     s_2d = umap.UMAP(
        #     n_components=2, n_neighbors=200, min_dist=0
        #     ).fit_transform(shap_values.values[:, :, 1])

        #     plt.scatter(
        #             X_2d[:, 0],
        #             X_2d[:, 1],
        #             s=0.2, 
        #             alpha=0.3,
        #             c='r' 
        #             )
        #     plt.title(f'UMAP projection of {window_size}', fontsize=24)

        #     plt.scatter(
        #             s_2d[:, 0],
        #             s_2d[:, 1],
        #             s=0.2, 
        #             alpha=0.3, 
        #             )
        #     plt.title(f'SHAP UMAP projection of {window_size}', fontsize=24)

        #     plt.show()    


In [ ]:
# def bootstrap (X, method, B, sigma_noise=None, no_bootstrap=False, random_seed=None, num_jobs=None, use_n_pcs=False, subsample=False, **kwargs):
#     '''
#     Creates n bootstrap data from X and creates a DR visualizastion for each of them.
    
#     Arguments:
#         See generate() for details
    
#     Returns:
#         X_embedded_list = list of the 2D DR visualization embeddings (numpy arrays)
#         bootstrap_indices_list = list of numpy arrays indicating the bootstrap row indices
#     '''

#     X_embedded_list = []
#     bootstrap_indices_list = []
    
#     # generate sequence of random states if random_seed is specified
#     if isinstance(random_seed, int):
#         seeded_rand1 = np.random.RandomState(random_seed)
#         random_seeded_sequence = seeded_rand1.randint(0,1e6,B)
#     else:
#         random_seeded_sequence = False
    
#     # bootstrap DR
#     if isinstance(num_jobs, int): # in parallel
#         result = Parallel(n_jobs=num_jobs)(delayed(run_one_bootstrap)(X, method, sigma_noise, no_bootstrap,
#                                            random_seeded_sequence, b, use_n_pcs, subsample, **kwargs) for b in tqdm(range(B)))
#         X_embedded_list = [x[0] for x in result]
#         bootstrap_indices_list = [x[1] for x in result]
#     else: # using only one core
#         for b in tqdm(range(B)):
#             X_embedded, boot_idxs = run_one_bootstrap(X, method, sigma_noise, no_bootstrap,
#                                            random_seeded_sequence, b, use_n_pcs, subsample, **kwargs)
#             X_embedded_list.append(X_embedded)
#             bootstrap_indices_list.append(boot_idxs)
    
#     return(X_embedded_list, bootstrap_indices_list)


# def run_one_bootstrap(X, method, sigma_noise=None, no_bootstrap=False, random_seeded_sequence=False, b=0, use_n_pcs=False, subsample=False, **kwargs):
#     '''
#     Method for generating one bootstrap X and one DR visualization of the bootstrap
    
#     Arguments:
#         random_seeded_sequence = array or list of random seeds to use in generating the bootstrap sample
#         b = integer specifying the index of random see in random_seeded_sequence to use for generating the bootstrap
#         See generate() for more details
    
#     Returns:
#         X_embedded = 2D DR visualization embedding (numpy array)
#         boot_idxs = numpy array indicating the bootstrap row indices
#     '''
#     # Create bootstrap X
#     if subsample is False:
#         if no_bootstrap is True: # don't bootstrap (will use intrinsic stochasticity of DR algorithm (if any) only)
#             boot_X = X.copy()
#             boot_idxs = np.arange(X.shape[0]) # set indices to be the original indices
#         elif random_seeded_sequence is not False: # use specified random_seeded_sequence to generate bootstrap X
#             seeded_rand2 = np.random.RandomState(random_seeded_sequence[b])
#             boot_idxs = seeded_rand2.randint(0,X.shape[0],X.shape[0])
#             boot_X = X.copy()[boot_idxs,:] 
#         else: # if no random_seeded_sequence, use default random process
#             boot_idxs = np.random.randint(0,X.shape[0],X.shape[0])
#             boot_X = X.copy()[boot_idxs,:]
#     # Subsample instead of bootstrapping
#     else:
#         if random_seeded_sequence is not False: # use specified random_seeded_sequence to generate subsample of X
#             seeded_rand2 = np.random.RandomState(random_seeded_sequence[b])
#             boot_idxs = seeded_rand2.choice(X.shape[0], subsample, replace=False)
#             boot_X = X.copy()[boot_idxs,:] 
#         else: # if no random_seeded_sequence, use default random process
#             boot_idxs = np.random.choice(X.shape[0], subsample, replace=False)
#             boot_X = X.copy()[boot_idxs,:]
        
#     # add Gaussian noise to alleviate duplicate issues if specified
#     if sigma_noise is not None:
#         boot_X += np.random.normal(0,sigma_noise,boot_X.shape)
        
#     # run TruncatedSVD if specified to do a first pass DR with PCA and take use_n_pcs top principal components for DR visualization
#     if use_n_pcs is not False:
#         boot_X = TruncatedSVD(n_components=use_n_pcs).fit_transform(boot_X)
    
#     # generate DR visualization embedding
#     X_embedded = dimensionality_reduction(boot_X, method, **kwargs)


#     return(X_embedded, boot_idxs)


# def generate(X, method, Y=None, B=0, sigma_noise=None, no_bootstrap=False, random_seed=None, save=False,
#              num_jobs=None, use_n_pcs=False, subsample=False, return_times=False, **kwargs):
#     '''
#     Main method for generating aligned bootstrap visualizations, which are the input elements for dynamic visualization.
    
#     Arguments:
#         X = (n x p) numpy array where rows are observations, columns are features
#         method = string, dimensionality reduction method to use; options include:
#             "tsne", "mds", "lle", "mlle", "isomap", "umap", "pca"
#         Y = pandas dataframe with same number of rows as X and columns containing relevant metadata to propagate to output
#         B = integer, number of bootstraps to generate; if B==0: generates only for the original X
#         sigma_noise = None or float, if float, adds zero-centered Gaussian noise to each bootstrap sample with standard deviation sigma_noise
#         no_bootstrap = True or False, whether to bootstrap sample for each iteration or not
#         random_seed = None or int, if int, uses that value to generate random sequence (i.e. bootstrap sequence will be the same)
#         save = False or str, if str, path to save resulting Pandas dataframe as CSV
#         num_jobs = None, -1, or >=1 int; if not None, runs multiprocessing with n_jobs, if n_jobs=-1, then uses all available
#         use_n_pcs = False or int, specifying to apply PCA and keep to use_n_pcs components to use for method
#         subsample = False or int, specifying whether to subsample INSTEAD OF bootstrapping with integer corresponding to size of subsample to take
#         return_times = True or False; if not False, returns a dictionary of run times broken down by components as the second output
    
#     Returns:
#         output = Pandas dataframe with "x1", "x2", "bootstrap_number", "original_index" as columns, along with columns of Y
#                     2D embedding is (x1, x2)
#     '''
#     # process results into dataframe
#     output = pd.DataFrame() # init df to be merged onto
    
#     # DR on original dataset
#     original_embedding = dimensionality_reduction(X, method, **kwargs) # Ex: can specify random_state in **kwargs to remove stochasticity
    
#     # reference points is the original dataset
#     points0 = np.hstack((original_embedding, np.zeros(original_embedding.shape[0]).reshape(original_embedding.shape[0],1)))# append uniform 3rd dimension

#     # add basic info
#     output["x1"] = points0[:,0]-np.mean(points0[:,0]) # center reference visualization at (0,0)
#     output["x2"] = points0[:,1]-np.mean(points0[:,1])
#     output["original_index"] = np.arange(len(points0[:,0]))
#     output["bootstrap_number"] = -1

#     # add metadata
#     if isinstance(Y, pd.DataFrame):
#         for col in Y.columns:
#             output[col] = Y[col].values
    
#     # keep track of run times
#     rt_dict = {}
    
#     # bootstrap
#     start_time = time.time()
#     if B > 0:
#         bootstrap_embedding_list, bootstrap_indices_list = bootstrap(X, method, B, sigma_noise, no_bootstrap, 
#                                                                     random_seed, num_jobs, use_n_pcs, subsample, **kwargs)
#     bootstrap_time = time.time() - start_time
#     rt_dict["bootstrapped_DR"] = bootstrap_time
    
#     # add bootstraps
#     start_time = time.time()
#     for i in range(len(bootstrap_embedding_list)):
        
#         new_df = pd.DataFrame() # new df to merge onto original df
        
#         points = bootstrap_embedding_list[i]
#         points = np.hstack((points, np.zeros(points.shape[0]).reshape(points.shape[0],1)))# append uniform 3rd dimension
#         boot_idxs = bootstrap_indices_list[i]
        
#         # rigid alignment w/ 3d rotation (Kabsch)
#         ref_points = points0[boot_idxs,:]
#         points[:,0] = points[:,0]-np.mean(points[:,0])
#         points[:,1] = points[:,1]-np.mean(points[:,1])
#         r = Rotation.align_vectors(ref_points, points)[0]
#         rpoints = r.apply(points)
        
#         # add basic info
#         new_df["x1"] = rpoints[:,0]
#         new_df["x2"] = rpoints[:,1]
#         new_df["original_index"] = boot_idxs
#         new_df["bootstrap_number"] = i
        
#         # add metadata
#         if isinstance(Y, pd.DataFrame):
#             for col in Y.columns:
#                 new_df[col] = Y[col].values[boot_idxs]
        
#         # merge to original dataframe
#         output = pd.concat([output, new_df], axis=0)
    
#     align_time = time.time() - start_time
#     rt_dict["alignment_DR"] = align_time
    
#     # save output
#     if save is not False:
#         output.to_csv(save, index=False)
    
#     if return_times is False:
#         return(output)
#     else:
#         return(output, rt_dict)
        